In [ ]:
!wget http://www.eraserbenchmark.com/zipped/movies.tar.gz
!tar -xzf movies.tar.gz
!mv movies/val.jsonl movies/dev.jsonl

In [ ]:
import sys
sys.path.append("../")
import json
from utils import load_flattened_documents
documents = load_flattened_documents('movies', docids=None)
def read_annotations(json_file, exp_split) :
    anns = [json.loads(line) for line in open(json_file)]
    for i, a in enumerate(anns) :
      
        doc_id = a['annotation_id']
        a["doc_id"] = doc_id
        a["annotation_id"] = exp_split + "_" + str(i)
        a['text'] = " ".join(documents[doc_id])
        a['label_id'] = a['classification']
        del a['classification']
        a['rationale'] = []
        for evgroup in a['evidences'] :
            for ev in evgroup :
                assert ev['docid'] == doc_id
                a['rationale'].append((ev['start_token'], ev['end_token']))
        a["exp_split"] = exp_split
        del a['evidences']
        del a['query_type']
        del a['query']
        
    return anns

In [ ]:
import os
os.makedirs('data/', exist_ok=True)

In [ ]:
import pandas as pd
for key in ['train','dev', 'test'] :
    
    df = pd.DataFrame(read_annotations("movies/" + key + ".jsonl", key))
    
    df["label"] = df.label_id.apply(lambda x : 0 if x == "NEG" else 1)
    
    df.to_csv("data/" + key + ".csv", index = False)
    

In [ ]:
df